In [ ]:
import subprocess
import os
import logging

# --- 1. Configuración del Logging para ver el progreso ---
# Esto mostrará mensajes informativos en la consola.
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%H:%M:%S'
)

# --- 2. Definición de Rutas del Proyecto ---
# Se asume que este script se ejecuta desde la raíz del proyecto.
project_root = os.getcwd() 
image_path_relative = os.path.join( "feature_importance.jpg")
image_path_absolute = os.path.join(project_root, image_path_relative)

# LaTeX necesita las barras hacia adelante (/), incluso en Windows.
image_path_latex = image_path_absolute.replace('\\', '/')

logging.info(f"Ruta de la imagen para LaTeX: {image_path_latex}")

# --- 3. Código LaTeX como una cadena de texto (con correcciones) ---
# Se ha actualizado el comando \includegraphics para usar la ruta absoluta.
latex_code = r"""
\documentclass{article}

% --- PAQUETES ---
\usepackage[utf8]{inputenc}
\usepackage[spanish]{babel}
\usepackage{graphicx}
\usepackage{url}
\usepackage[margin=1in]{geometry}
\usepackage{booktabs}
\usepackage{hyperref}
\usepackage[numbers, sort&compress]{natbib}
\usepackage{float} % <--- Paquete para forzar la posición con [H]

% --- METADATOS DEL DOCUMENTO ---
\title{Alerta de Bajo Peso al Nacer con Machine Learning}
\author{
  Pedro Emilse Casaverde Ayma \\
  \and
  Sherly Thalia Salazar Estela
}
\date{Curso: IA Fundamentals}

% --- Corrección de "Cuadro" a "Tabla" ---
\addto\captionsspanish{\renewcommand{\tablename}{Tabla}}

\begin{document}

\maketitle

% --- RESUMEN ---
\begin{abstract}
El bajo peso al nacer (BPN) es un problema de salud pública crítico en Perú, asociado a una elevada morbimortalidad infantil. Este estudio propone un modelo de \textit{machine learning} para la detección temprana de riesgo de BPN, utilizando datos de la Encuesta Demográfica y de Salud Familiar (ENDES). La metodología abarcó la combinación de datos de los años 2023 y 2024, la creación de características nuevas como la distancia entre la vivienda y la capital provincial, y la evaluación comparativa de cuatro algoritmos: Balanced Random Forest, LightGBM, Regresión Logística y XGBoost. Los resultados identifican al modelo XGBoost como el de mejor rendimiento, con un AUC-ROC del 70.31\%. Las variables más predictivas incluyen el número de controles prenatales, la tenencia de carnet de salud y el consumo de suplementos de hierro. El modelo resultante representa una herramienta potencialmente valiosa para la focalización de intervenciones en salud pública.
\end{abstract}

% --- SECCIONES DEL PAPER ---
\section{Introducción}
El peso al nacer es un indicador fundamental de la salud neonatal. Su disminución por debajo de los 2500 gramos, conocido como Bajo Peso al Nacer (BPN), constituye uno de los principales predictores de mortalidad infantil y complicaciones de salud a largo plazo \cite{benjumea2009, demorais2025}. En el contexto peruano, el BPN sigue siendo una preocupación prevalente, con tasas que varían significativamente por región \cite{ticona2012}. La identificación temprana de gestantes en riesgo es, por tanto, una prioridad para optimizar la asignación de recursos y aplicar intervenciones preventivas de manera eficaz.

El desafío radica en la naturaleza multifactorial del BPN. Mientras los estudios tradicionales han empleado métodos como la regresión logística para identificar factores de riesgo \cite{ticona2012}, estos enfoques pueden tener limitaciones para capturar relaciones no lineales. Los avances en \textit{machine learning} (ML) ofrecen una alternativa robusta para el modelado predictivo en salud, capaces de descubrir patrones sutiles que no son evidentes con técnicas convencionales \cite{ahmadi2017, demorais2025}.

En este trabajo se presenta el desarrollo y validación de un modelo predictivo de BPN utilizando datos públicos de la Encuesta Demográfica y de Salud Familiar (ENDES) de Perú. Se comparó el rendimiento de cuatro modelos de clasificación y se realizó ingeniería de características para crear una variable de alto valor informativo que mide la distancia entre la vivienda y la capital de la provincia, como un proxy de accesibilidad a servicios.

La contribución principal es un modelo validado basado en XGBoost que alcanza un AUC-ROC del 70.31\%. El análisis de importancia de características revela que el número de controles prenatales, la tenencia de carnet de salud y el consumo de suplementos de hierro son los factores más determinantes.

La implementación completa de este proyecto está disponible en GitHub: \url{https://github.com/PedroCasaverde/low-birth-weight-alert}.

\section{Trabajos Relacionados}
La predicción del BPN mediante técnicas computacionales es un área de investigación activa. Enfoques iniciales en el contexto peruano, como el de Ticona y Huanco (2012), utilizaron la regresión logística en Tacna para demostrar que condiciones socioeconómicas y patologías maternas estaban asociadas al BPN. Sin embargo, la literatura reciente ha explorado la superioridad de los modelos de ML. Por ejemplo, Ahmadi et al. (2017) concluyeron que Random Forest (RF) ofrecía una predicción superior a la regresión logística (AUC de 93\% vs 89\%) gracias a su capacidad para modelar interacciones complejas.

Dentro de los modelos basados en árboles, diversas variantes han sido aplicadas. Ramos-Purihuaman y Santos-Rosales (2024) utilizaron un árbol de decisión en un hospital peruano para predecir la valoración del recién nacido a partir del estado nutricional materno. De manera similar, Benjumea et al. (2009) emplearon árboles de clasificación (CART) y destacaron la importancia de los cambios en medidas antropométricas entre trimestres, variables también consideradas en nuestro estudio.

Estudios más exhaustivos, como el de de Morais et al. (2025) en Brasil, han evaluado un espectro más amplio de modelos de ensamble, incluyendo XGBoost y LightGBM. Su trabajo resalta la importancia de un preprocesamiento cuidadoso y corrobora que las características sociodemográficas y el historial gestacional son las más influyentes. Nuestro proyecto se alinea con esta última corriente al emplear datos de una encuesta nacional y comparar modelos de ensamble avanzados.

\section{Propuesta Metodológica}
El flujo de trabajo propuesto para el desarrollo del modelo predictivo de BPN se estructuró en cinco fases principales.

\subsection{Adquisición y Fuentes de Datos}
El modelo se nutre de dos fuentes de datos públicas:
\begin{itemize}
    \item \textbf{Encuesta Demográfica y de Salud Familiar (ENDES):} Microdatos de los años 2023 y 2024.
    \item \textbf{Datos de Ubigeo y Socioeconómicos:} Un archivo a nivel provincial con indicadores como el IDH y tasas de pobreza.
\end{itemize}

\subsection{Unificación y Preprocesamiento}
En esta fase se unificaron los datos de los 13 módulos de la ENDES de ambos años. Los conjuntos de datos se combinaron para formar una única base de datos maestra, gestionando valores ausentes mediante imputación (mediana para variables numéricas, moda para categóricas).

\subsection{Ingeniería de Características}
Para mejorar la capacidad predictiva se creó una nueva variable:
\begin{itemize}
    \item \textbf{Distancia a la Capital de la Provincia:} Mide la distancia en kilómetros entre la vivienda de la encuestada y la capital de su provincia, como un proxy de la accesibilidad a servicios.
\end{itemize}

\subsection{Entrenamiento y Evaluación de Modelos}
\begin{itemize}
    \item \textbf{Preparación:} La variable objetivo, \texttt{bajo\_peso\_nacimiento}, fue definida como binaria (1 si el peso es $<$ 2500g) \cite{ramos2024}. Se excluyeron variables post-parto para evitar fuga de datos.
    \item \textbf{Modelos Seleccionados:} Se evaluaron: Regresión Logística \cite{ticona2012}, Balanced Random Forest (BRF), LightGBM y XGBoost \cite{demorais2025}.
    \item \textbf{Evaluación:} Se utilizó una validación cruzada de 5 pliegues, priorizando la métrica AUC-PR por su robustez en escenarios con clases desbalanceadas.
\end{itemize}

\section{Experimentos y Resultados}
El experimento principal consistió en la evaluación comparativa de los cuatro modelos de clasificación mencionados. El objetivo era determinar qué algoritmo generalizaba mejor para la tarea de predicción de BPN en el conjunto de datos de la ENDES.

\begin{table}[H] % <--- Posicionamiento forzado con [H]
\centering
\caption{Resumen de Métricas de Rendimiento de los Modelos (Validación Cruzada)}
\label{tab:metrics}
\begin{tabular}{lccccccc}
\toprule
\textbf{Modelo} & \textbf{Precision} & \textbf{Recall} & \textbf{Specificity} & \textbf{F1} & \textbf{AUC\_ROC} & \textbf{AUC\_PR} & \textbf{MCC} \\
\midrule
Balanced RF & 14.59\% & 36.86\% & 85.34\% & 20.36\% & 68.68\% & 13.78\% & 0.148127 \\
LightGBM & 7.43\% & 55.71\% & 54.66\% & 12.86\% & 56.17\% & 6.92\% & 0.052130 \\
Regresión Logística & 8.71\% & 54.58\% & 63.4\% & 15.0\% & 61.53\% & 8.69\% & 0.088403 \\
\textbf{XGBoost} & \textbf{16.35\%} & \textbf{34.04\%} & \textbf{88.66\%} & \textbf{21.86\%} & \textbf{70.31\%} & \textbf{15.03\%} & \textbf{0.163084} \\
\bottomrule
\end{tabular}
\end{table}

La Tabla~\ref{tab:metrics} resume las métricas de rendimiento promedio obtenidas. Los resultados muestran que \textbf{XGBoost} es el modelo con el mejor rendimiento general. Alcanzó el valor más alto en AUC-ROC (70.31\%), AUC-PR (15.03\%) y el Coeficiente de Correlación de Matthews (MCC) (0.163), lo que indica una mayor capacidad discriminativa. Aunque su Recall (34.04\%) no fue el más alto, su Precisión (16.35\%) fue superior, lo que sugiere que sus predicciones positivas son más confiables.

Es importante contextualizar estos resultados. Si bien un Recall del 34.04\% podría considerarse bajo en un problema de clasificación binaria estándar, el objetivo principal de este modelo no es la clasificación categórica, sino la estratificación de riesgo. El resultado más valioso del modelo es la \textbf{probabilidad} de BPN que arroja para cada gestante. Esta probabilidad permitirá crear un sistema de alertas escalonado (por ejemplo: riesgo bajo $<$50\%, medio 50-75\%, y alto $>$75\%), permitiendo a los profesionales de la salud focalizar recursos y atención de manera más eficiente. Para esta aplicación, un modelo con un AUC-ROC superior, como el de XGBoost, es preferible, ya que demuestra una mejor capacidad para ordenar correctamente a las pacientes según su nivel de riesgo.

\subsection{Análisis de Importancia de Características}
El análisis del modelo XGBoost ganador reveló los factores más influyentes en la predicción del BPN, como se observa en la Figura~\ref{fig:importance}.

\begin{figure}[H] % <--- Posicionamiento forzado con [H]
    \centering
    \caption{Importancia de las Características del Modelo XGBoost.}
    \label{fig:importance}
    \includegraphics[width=\columnwidth]{""" + image_path_latex + r"""}
\end{figure}

Los predictores más determinantes, en orden de importancia, fueron:
\begin{enumerate}
    \item \textbf{Nº de Controles Prenatales}: El número de controles prenatales. Este hallazgo es consistente con la literatura, donde un seguimiento adecuado permite detectar y manejar riesgos a tiempo \cite{ticona2012}.
    \item \textbf{Tiene Carnet de Salud}: La posesión de un carnet de salud, un indicador del vínculo de la madre con el sistema sanitario.
    \item \textbf{Consumió Suplemento de Hierro}: El consumo de suplementos, una intervención directa sobre la nutrición materna y la prevención de anemia \cite{benjumea2009}.
    \item \textbf{Macrorregión (INEI)}: Esta variable resalta profundas disparidades regionales en los resultados de salud \cite{ticona2012}.
    \item \textbf{Deseo de Tener más Hijos}: Un posible proxy de la planificación familiar y las condiciones socioeconómicas.
\end{enumerate}

\section{Conclusiones}
Este estudio demuestra la viabilidad y eficacia de utilizar técnicas de machine learning para desarrollar un modelo de alerta temprana de bajo peso al nacer en el contexto peruano. El modelo basado en XGBoost logró un rendimiento predictivo superior (AUC-ROC de 70.31\%), confirmando que los modelos de \textit{gradient boosting} son una herramienta poderosa.

El análisis de características ratifica la importancia de factores asociados al comportamiento y acceso al sistema de salud. El número de controles prenatales, la tenencia de carnet de salud y el consumo de suplementos emergieron como los predictores más potentes, reforzando la necesidad de fortalecer las políticas públicas orientadas a garantizar la adherencia al cuidado prenatal.

La relevancia de variables como la macrorregión confirma que las desigualdades geográficas y socioeconómicas son un factor determinante en los resultados perinatales en Perú. Las intervenciones futuras podrían beneficiarse de un enfoque territorial para priorizar a las poblaciones más vulnerables.

Finalmente, el modelo desarrollado sienta las bases para una herramienta práctica de soporte a la decisión. Su implementación podría ayudar a los profesionales de la salud a estratificar el riesgo de las gestantes de manera más precisa, permitiendo una asignación de recursos más eficiente.

% --- BIBLIOGRAFÍA ---
\bibliographystyle{plainnat}
\begin{thebibliography}{5}

\bibitem{ahmadi2017}
P. Ahmadi, H. Alavi Majd, S. Khodakarim, L. Tapak, N. Kariman, P. Amini, \& F. Pazhuheian. (2017).
Prediction of low birth weight using Random Forest: A comparison with Logistic Regression.
\textit{Journal of Paramedical Sciences, 8}(3), 36-43.

\bibitem{benjumea2009}
M. V. Benjumea Rincón, J. Bacallao Gallestey, \& R. Jiménez. (2009).
La predicción del bajo peso y del peso insuficiente al nacer mediante la antropometría materna.
\textit{Hacia la Promoción de la Salud, 14}(1), 35-53.

\bibitem{demorais2025}
F. L. de Morais, E. da Silva Rocha, G. Masson, D. C. do Nascimento Filho, K. M. Mendes, R. A. de Sousa Dourado, W. Brandao Neto, \& P. T. Endo. (2025).
Utilization of tree-based machine learning models for predicting low birth weight cases.
\textit{BMC Pregnancy and Childbirth, 25}(207), 1-32.

\bibitem{ramos2024}
K. C. Ramos-Purihuaman, \& Y. R. Santos-Rosales. (2024).
Predicción de la valoración del recién nacido según el estado nutricional materno: modelo basado en árboles de decisión.
\textit{Revista de Obstetricia y Ginecología de Venezuela, 84}(2), 115-123.

\bibitem{ticona2012}
M. Ticona Rendón, \& D. Huanco Apaza. (2012).
Factores de riesgo del peso insuficiente al nacer, en el hospital Hipólito Unanue de Tacna, 2001-2010.
\textit{Revista Peruana de Ginecología y Obstetricia, 58}(3), 169-175.

\end{thebibliography}

\end{document}
"""


# Nombre del archivo sin extensión
filename = "proyecto_final_IA_fundamentals"

# --- 4. Proceso de Generación de PDF ---
logging.info("Iniciando la generación del PDF...")

# Escribir el código a un archivo .tex
tex_filepath = f"{filename}.tex"
try:
    with open(tex_filepath, "w", encoding="utf-8") as f:
        f.write(latex_code)
    logging.info(f"Archivo '{tex_filepath}' creado correctamente.")
except Exception as e:
    logging.error(f"No se pudo escribir el archivo .tex: {e}")
    exit()

# Compilar el .tex a .pdf usando pdflatex
try:
    # Primera compilación
    logging.info("Iniciando primera compilación con pdflatex...")
    result1 = subprocess.run(['pdflatex', '-interaction=nonstopmode', tex_filepath], check=True, capture_output=True, text=True, encoding='latin1')
    logging.info("Primera compilación completada.")
    
    # Segunda compilación para resolver referencias cruzadas
    logging.info("Iniciando segunda compilación para referencias...")
    result2 = subprocess.run(['pdflatex', '-interaction=nonstopmode', tex_filepath], check=True, capture_output=True, text=True, encoding='latin1')
    logging.info("Segunda compilación completada.")

    logging.info(f" ¡PDF '{filename}.pdf' generado con éxito!")

except FileNotFoundError:
    logging.error(" Error: 'pdflatex' no se encontró. Asegúrate de que tienes una distribución de LaTeX instalada y en el PATH del sistema.")
except subprocess.CalledProcessError as e:
    logging.error(f" Error durante la compilación de LaTeX. Revisa el archivo '{filename}.log' para más detalles.")
    # Imprime la salida de error de LaTeX para facilitar la depuración
    logging.error(f"--- Salida del Compilador (stdout) ---\n{e.stdout}")
    logging.error(f"--- Salida de Error del Compilador (stderr) ---\n{e.stderr}")
finally:
    # 5. (Opcional) Limpiar archivos auxiliares generados por LaTeX
    logging.info("Limpiando archivos auxiliares (.aux, .log, .tex)...")
    for ext in ['.aux', '.log', '.tex']:
        try:
            if os.path.exists(f"{filename}{ext}"):
                os.remove(f"{filename}{ext}")
        except Exception as e:
            logging.warning(f"No se pudo eliminar el archivo {filename}{ext}: {e}")
    logging.info("Proceso finalizado.")
